## 准备数据

In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [24]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [25]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28*28, 128], mean=0.0, stddev=0.1), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([128]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([128, 10], mean=0.0, stddev=0.1), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)
    
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [26]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [27]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.5042467 ; accuracy 0.070766665
epoch 1 : loss 2.484389 ; accuracy 0.07271667
epoch 2 : loss 2.465681 ; accuracy 0.07416666
epoch 3 : loss 2.4479885 ; accuracy 0.07631667
epoch 4 : loss 2.4311957 ; accuracy 0.07915
epoch 5 : loss 2.4151995 ; accuracy 0.081933334
epoch 6 : loss 2.3999124 ; accuracy 0.08471667
epoch 7 : loss 2.3852594 ; accuracy 0.087866664
epoch 8 : loss 2.3711727 ; accuracy 0.09115
epoch 9 : loss 2.3575976 ; accuracy 0.09495
epoch 10 : loss 2.3444836 ; accuracy 0.098566666
epoch 11 : loss 2.3317885 ; accuracy 0.10243333
epoch 12 : loss 2.3194723 ; accuracy 0.107433334
epoch 13 : loss 2.3075018 ; accuracy 0.11186667
epoch 14 : loss 2.2958455 ; accuracy 0.11688333
epoch 15 : loss 2.284478 ; accuracy 0.12213334
epoch 16 : loss 2.2733734 ; accuracy 0.1281
epoch 17 : loss 2.2625124 ; accuracy 0.13471666
epoch 18 : loss 2.2518756 ; accuracy 0.14086667
epoch 19 : loss 2.2414463 ; accuracy 0.14736667
epoch 20 : loss 2.2312093 ; accuracy 0.1536
epoch 21 : loss 2